#1

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
%reset-f

In [27]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import math
import copy

In [28]:
def hypo(inst,w,n):
  p = np.dot(inst[:,0:n],w.T)
  h = 1/(1+np.exp(-p.astype(np.float32)))
  h[h<=0.5] = -1
  h[h>0.5] = 1
  return h

def update(inst,w,n,m,a):
  mistakes = 0
  h = hypo(inst,w,n)
  for i in range(m):
    if h[i] != inst[i,n]:
      mistakes +=1
      for j in range(n):
        w[j] = w[j] + a*inst[i,-1]*inst[i,j]
  return mistakes,w

def errors(inst,w,n,m):
  h = hypo(inst,w,n)
  mistakes = 0
  for i in range(m):
    if h[i] != inst[i,n]:
      mistakes += 1
  return mistakes

def perfor(mat):
  accuracy = (mat[0,0]+mat[1,1]/mat[0,0]+mat[0,1]+mat[1,0]+mat[1,1])
  sensi = (mat[1,1]/(mat[1,1]+mat[1,0]))
  speci = (mat[0,0]/(mat[0,1]+mat[0,0]))
  print('Accuracy is ' , end = ' ')
  print(accuracy)
  print('sensitivity is ',end = ' ')
  print(sensi)
  print('specificity is ',end=' ')
  print(speci)

In [29]:
df = pd.read_excel('/content/drive/MyDrive/ML/data55.xlsx')
inst = df.to_numpy()
m = len(inst[:,0]) #number of instances
n = len(inst[0,:]) #number of features
inst = np.append(np.ones((m,1)),inst,axis=1)
for i in range(1,n,1):
  inst[:,i] = (inst[:,i] - inst[:,i].mean())/inst[:,i].std() #normalizing the data
np.random.shuffle(inst)
#splitting data into training,testing and validation
inst_train = inst[0:int(0.7*m),:]
inst_test = inst[int(0.7*m):int(0.9*m),:]
inst_val = inst[int(0.9*m):m,:]

inst_train[:,-1][inst_train[:,-1]==0] = -1
inst_val[:,-1][inst_val[:,-1] == 0] = -1
inst_test[:,-1][inst_test[:,-1]==0] = -1

m_train = len(inst_train[:,0])
m_val = len(inst_val[:,0]) 
m_test = len(inst_test[:,0])

w_optimal = np.zeros(n)
iter = 500
alpha = np.linspace(0.001,0.01,10)
b_min = m+1

for j in range(len(alpha)):
  w = np.zeros(n)
  for ite in range(iter):
    mistakes = 0
    mistakes, w= update(inst_train,w,n,m_train,alpha[j])
    if(mistakes == 0):
      break
  mis_val = errors(inst_val,w,n,m_val)
  if(mis_val<b_min):
    b_min = mis_val
    w_optimal = w

In [30]:
h = hypo(inst_test,w_optimal,n)
confmat = pd.crosstab(inst_test[:,-1], h)
confmat = np.asarray(confmat)
perfor(confmat)

Accuracy is  42.526315789473685
sensitivity is  0.5263157894736842
specificity is  0.8260869565217391
